# Identifying which food distribution sector produces the most food waste

This project hopes to tabulate and model how much Food Loss and Waste (FLW) is produced throughout the Food Supply Chain (FSC). Data collected will be used to train a linear regression model to predict food loss for each FSC stage. The results will show at which stages of the FSC produces the most and least amount of waste.

## Importing Libraries

In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


## Loading the Dataset

The data used in this project came from [FAO's Food Loss and Waste Database](https://www.fao.org/platform-food-loss-waste/flw-data/en/). It is a database that shows how much food was wasted and in what stage it was wasted.

In [2]:
# read the data from the csv into the dataframe
df = pd.read_csv("data.csv", header=0)

# print the column names
print('---ORIGINAL DATASET---\n')
print('COLUMNS')
print(df.columns)
print()

# print the shape
print('SHAPE')
print(df.shape)

# show the first 5 rows of the dataframe
df.head()

---ORIGINAL DATASET---

COLUMNS
Index(['m49_code', 'country', 'region', 'cpc_code', 'commodity', 'year',
       'loss_percentage', 'loss_percentage_original', 'loss_quantity',
       'activity', 'food_supply_stage', 'treatment', 'cause_of_loss',
       'sample_size', 'method_data_collection', 'reference', 'url', 'notes'],
      dtype='object')

SHAPE
(32580, 18)


,m49_code,country,region,cpc_code,commodity,year,loss_percentage,loss_percentage_original,loss_quantity,activity,food_supply_stage,treatment,cause_of_loss,sample_size,method_data_collection,reference,url,notes
0,104,Myanmar,NaN,0142,"Groundnuts, excluding shelled",2009,5.22,5.22%,68100,NaN,Whole supply chain,NaN,NaN,NaN,FAO's annual Agriculture Production Questionna...,FAO Sources,NaN,NaN
1,104,Myanmar,NaN,0142,"Groundnuts, excluding shelled",2008,5.43,5.43%,65240,NaN,Whole supply chain,NaN,NaN,NaN,FAO's annual Agriculture Production Questionna...,FAO Sources,NaN,NaN
2,104,Myanmar,NaN,0142,"Groundnuts, excluding shelled",2007,5.61,5.61%,61080,NaN,Whole supply chain,NaN,NaN,NaN,FAO's annual Agriculture Production Questionna...,FAO Sources,NaN,NaN
3,104,Myanmar,NaN,0142,"Groundnuts, excluding shelled",2006,5.40,5.4%,55270,NaN,Whole supply chain,NaN,NaN,NaN,FAO's annual Agriculture Production Questionna...,FAO Sources,NaN,NaN
4,104,Myanmar,NaN,0142,"Groundnuts, excluding shelled",2005,5.00,5%,51970,NaN,Whole supply chain,NaN,NaN,NaN,FAO's annual Agriculture Production Questionna...,FAO Sources,NaN,NaN


## Preprocessing
This secttion is divided further into
- Data Representation
- Data Cleaning
- Data Transformation
    - Encoding
    - Scaling
- Creating Train and Test Data

#### Data Representation
Extract the necessary columns for this project (commodity, loss_percentage, and food_supply_stage)

In [3]:
# extract necessary columns
df = df[["commodity", "food_supply_stage", "loss_percentage"]]
df.head()

,commodity,food_supply_stage,loss_percentage
0,"Groundnuts, excluding shelled",Whole supply chain,5.22
1,"Groundnuts, excluding shelled",Whole supply chain,5.43
2,"Groundnuts, excluding shelled",Whole supply chain,5.61
3,"Groundnuts, excluding shelled",Whole supply chain,5.40
4,"Groundnuts, excluding shelled",Whole supply chain,5.00


In [4]:
# use groupby on commodity and food_supply_stage
# since there could be same commodity and food_supply_stage with different loss_percentage
df = df.groupby(["commodity", "food_supply_stage"]).mean().reset_index()
df.head()

,commodity,food_supply_stage,loss_percentage
0,"Almonds, in shell",Harvest,1.000000
1,"Almonds, in shell",Retail,1.000000
2,"Almonds, in shell",Storage,3.000000
3,"Almonds, in shell",Transport,1.000000
4,"Almonds, in shell",Whole supply chain,4.217368


#### Data Cleaning
Remove rows with missing values

In [5]:
# print how many missing values on each column
print("Before removing rows with missing values")
print(df.isna().sum())
print(df.shape)
print()

# remove the rows with missing values
df = df.dropna()

# print again how many missing values on each column
print("After removing rows with missing values")
print(df.isna().sum())
print(df.shape)


Before removing rows with missing values
commodity            0
food_supply_stage    0
loss_percentage      0
dtype: int64
(842, 3)

After removing rows with missing values
commodity            0
food_supply_stage    0
loss_percentage      0
dtype: int64
(842, 3)


#### Data Transformation
- Encoding
- Scaling

##### Encoding
One-hot encoding is used for the food_supply_stage feature while label encoding is used for the commodity feature

In [6]:
# get all categorical and numerical columns
data_column_category = df.select_dtypes(exclude=[np.number]).columns
data_column_number = df.select_dtypes(include=[np.number]).columns

In [7]:
# perform label encoding
label_encode_columns = ["commodity"]
label_encoder = LabelEncoder()
df_before_encoding = df.copy()

for i in data_column_category:
    df.loc[:,i] = label_encoder.fit_transform(df[i])

df.head()

,commodity,food_supply_stage,loss_percentage
0,0,5,1.000000
1,0,12,1.000000
2,0,14,3.000000
3,0,16,1.000000
4,0,17,4.217368


In [8]:
# perform one-hot encoding
onehot_encode_columns = ["food_supply_stage"]
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(df[onehot_encode_columns])
onehot_encoded_frame = pd.DataFrame(onehot_encoded, columns = onehot_encoder.get_feature_names(onehot_encode_columns))
df_onehot_getdummies = pd.get_dummies(df_before_encoding[onehot_encode_columns], prefix=onehot_encode_columns)
data_onehot_encoded_data = pd.concat([df[label_encode_columns], df_before_encoding[data_column_number], df_onehot_getdummies],axis = 1)
df = data_onehot_encoded_data
df.head()

,commodity,loss_percentage,food_supply_stage_Distribution,food_supply_stage_Export,food_supply_stage_Farm,food_supply_stage_Food Services,food_supply_stage_Grading,food_supply_stage_Harvest,food_supply_stage_Households,food_supply_stage_Market,...,food_supply_stage_Post-harvest,food_supply_stage_Pre-harvest,food_supply_stage_Processing,food_supply_stage_Retail,food_supply_stage_Stacking,food_supply_stage_Storage,food_supply_stage_Trader,food_supply_stage_Transport,food_supply_stage_Whole supply chain,food_supply_stage_Wholesale
0,0,1.000000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1.000000,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,3.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,4.217368,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


##### Scaling
MinMaxScaler method will be used

In [9]:
# perform min-max scaler
norm_scale = MinMaxScaler().fit_transform(df)
scaled_frame = pd.DataFrame(norm_scale,columns=df.columns)
df = scaled_frame
df.head()

,commodity,loss_percentage,food_supply_stage_Distribution,food_supply_stage_Export,food_supply_stage_Farm,food_supply_stage_Food Services,food_supply_stage_Grading,food_supply_stage_Harvest,food_supply_stage_Households,food_supply_stage_Market,...,food_supply_stage_Post-harvest,food_supply_stage_Pre-harvest,food_supply_stage_Processing,food_supply_stage_Retail,food_supply_stage_Stacking,food_supply_stage_Storage,food_supply_stage_Trader,food_supply_stage_Transport,food_supply_stage_Whole supply chain,food_supply_stage_Wholesale
0,0.0,0.015009,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.015009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.046279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.015009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.065312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### Show the preprocessed Dataset

In [10]:
# print the column names
print('---PREPROCESSED DATASET---\n')
print('COLUMNS')
print(df.columns)
print()

# print the shape
print('SHAPE')
print(df.shape)

# show the first 5 rows of the dataframe
df.head()

---PREPROCESSED DATASET---

COLUMNS
Index(['commodity', 'loss_percentage', 'food_supply_stage_Distribution',
       'food_supply_stage_Export', 'food_supply_stage_Farm',
       'food_supply_stage_Food Services', 'food_supply_stage_Grading',
       'food_supply_stage_Harvest', 'food_supply_stage_Households',
       'food_supply_stage_Market', 'food_supply_stage_Packing',
       'food_supply_stage_Post-harvest', 'food_supply_stage_Pre-harvest',
       'food_supply_stage_Processing', 'food_supply_stage_Retail',
       'food_supply_stage_Stacking', 'food_supply_stage_Storage',
       'food_supply_stage_Trader', 'food_supply_stage_Transport',
       'food_supply_stage_Whole supply chain', 'food_supply_stage_Wholesale'],
      dtype='object')

SHAPE
(842, 21)


,commodity,loss_percentage,food_supply_stage_Distribution,food_supply_stage_Export,food_supply_stage_Farm,food_supply_stage_Food Services,food_supply_stage_Grading,food_supply_stage_Harvest,food_supply_stage_Households,food_supply_stage_Market,...,food_supply_stage_Post-harvest,food_supply_stage_Pre-harvest,food_supply_stage_Processing,food_supply_stage_Retail,food_supply_stage_Stacking,food_supply_stage_Storage,food_supply_stage_Trader,food_supply_stage_Transport,food_supply_stage_Whole supply chain,food_supply_stage_Wholesale
0,0.0,0.015009,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.015009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.046279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.015009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.065312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### Creating Train and Test Data
The dataset for each FSC stage will be divided into 80:20 ratio for training and testing data

In [11]:
# store all feature columns in 'X' and the target feature in 'y'
X = df.drop('loss_percentage', axis=1)
y = df['loss_percentage']

# split the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("X_train : ",X_train.shape)
print("X_test : ",X_test.shape)
print("y_train : ",y_train.shape)
print("y_test : ",y_test.shape)

X_train :  (673, 20)
X_test :  (169, 20)
y_train :  (673,)
y_test :  (169,)
